# Prosperity trading algorithm

## Set up

Import all nessesary dependencies and create an example `TradingState` object.

In [1]:
from datamodel import Listing, Trade, OrderDepth, TradingState, Observation

timestamp = 1000

listings = {
	"AMETHYSTS": Listing(
		symbol="AMETHYSTS", 
		product="AMETHYSTS", 
		denomination="SEASHELLS"
	),
	"STARFRUIT": Listing(
		symbol="STARFRUIT", 
		product="STARFRUIT",
		denomination="SEASHELLS"
	),
}

order_depths = {
	"AMETHYSTS": OrderDepth(
		buy_orders={10: 7, 9: 5},
		sell_orders={11: -4, 12: -8}
	),
	"STARFRUIT": OrderDepth(
		buy_orders={142: 3, 141: 5},
		sell_orders={144: -5, 145: -8}
	),	
}

own_trades = {
	"AMETHYSTS": [],
	"STARFRUIT": []
}

market_trades = {
	"AMETHYSTS": [
		Trade(
			symbol="AMETHYSTS",
			price=11,
			quantity=4,
			buyer="",
			seller="",
			timestamp=900
		)
	],
	"STARFRUIT": []
}

position = {
	"AMETHYSTS": 3,
	"STARFRUIT": -5
}

observations = Observation(plainValueObservations={},
                           conversionObservations={})
traderData = ""

state = TradingState(
	traderData,
	timestamp,
    listings,
	order_depths,
	own_trades,
	market_trades,
	position,
	observations
)

## Trader class

Test trader class locally

In [2]:
from JM.trader_jm import Trader

trader = Trader()
result, conversions, traderData = trader.run(state)

[[1000,"",[["AMETHYSTS","AMETHYSTS","SEASHELLS"],["STARFRUIT","STARFRUIT","SEASHELLS"]],{"AMETHYSTS":[{"10":7,"9":5},{"11":-4,"12":-8}],"STARFRUIT":[{"142":3,"141":5},{"144":-5,"145":-8}]},[],[["AMETHYSTS",11,4,"","",900]],{"AMETHYSTS":3,"STARFRUIT":-5},[{},{}]],[["AMETHYSTS",9998,13],["AMETHYSTS",9996,3],["AMETHYSTS",10002,-18],["AMETHYSTS",10004,-4]],0,"{\"AMETHYSTS\": {\"position_limit\": 20, \"buy_price\": [{\"py/tuple\": [9998, 0.8]}, {\"py/tuple\": [9996, 0.2]}], \"sell_price\": [{\"py/tuple\": [10002, 0.8]}, {\"py/tuple\": [10004, 0.2]}]}, \"STARFRUIT\": {\"position_limit\": 20}}",""]
